In [26]:
import pandas as pd
import os 
import os.path as osp
import glob

In [21]:
os.getcwd()

'D:\\Project- Electro\\social_behavior_electro\\data_organization\\python'

In [22]:
# folder_name = 'D://Project- Electro//social_behavior_electro//analysis//affiliative//population analysis results'
# sociability = 'affiliative'
sociability = 'aversive'

folder_name = osp.join('..','..','analysis',sociability,'population analysis results')
df_dict_list = glob.glob(osp.join(folder_name, 'Population analysis results for LFP - Mean power*'))
df_dict_list

['..\\..\\analysis\\aversive\\population analysis results\\Population analysis results for LFP - Mean power_avv_111_2.xlsx',
 '..\\..\\analysis\\aversive\\population analysis results\\Population analysis results for LFP - Mean power_avv_12_15.xlsx',
 '..\\..\\analysis\\aversive\\population analysis results\\Population analysis results for LFP - Mean power_avv_13_112.xlsx',
 '..\\..\\analysis\\aversive\\population analysis results\\Population analysis results for LFP - Mean power_avv_14_16.xlsx']

In [23]:
def fix_file_paths(df_files):
    
    df_files = df_files.rename(columns={df_files.columns[0]:'timestamps', df_files.columns[0]:'filename', })
    
    return df_files


In [24]:
# Get number of session per group
sheet_name = 'Uploaded files'
df_files = fix_file_paths(df_dict_list[sheet_name])
max_len = len(df_files)

TypeError: list indices must be integers or slices, not str

In [ ]:
substrings_to_remove = ['Mean','Before','After', 'During','First', 'Second', 'Area']

df_all_list = []
for df_name in df_dict_list:
    df_dict = pd.read_excel(df_name, None)
    
    file_sheet_name = 'Uploaded files'
    filenames = fix_file_paths(df_dict[sheet_name])
    filenames = filenames. iloc[:,0]
    max_len = len(filenames)
    
    

    for sheet, df in df_dict.items():
#         if len(df.columns) > 1:
#             continue
        if "All" in sheet:
            continue
            
        if "Uploaded" in sheet:
            continue
        
        
        df_temp = pd.DataFrame()
        area_name = sheet
        for sub_string_to_remove in substrings_to_remove:
            area_name = area_name.replace(sub_string_to_remove,'')
        area_name = str(area_name)

        vals = df.iloc[:max_len]
        
        if "Before" in sheet:
            stage = "1_Before"
        elif "During" in sheet:
            stage = "2_During"
        elif "After" in sheet:
            stage = "3_After"
        else:
            print(sheet)
            raise
            
        if "First" in sheet:
            freq = '4-12'
        elif "Second" in sheet:
            freq = "30-80"
        else:
            raise
        
        df_temp['values'] = vals
        df_temp['area'] = area_name
        df_temp['stage'] = stage
        df_temp['freq'] = freq
        df_temp['filenames'] = filenames
        
        df_all_list.append(df_temp)
df_all = pd.concat(df_all_list)          

In [ ]:
df = df_all.copy()
# Create a pivot table to unstack 'freq' and 'stage' columns
pivot_df = pd.pivot_table(df, values='values', index=['filenames'], columns=['area','freq','stage'])

# Reset the index to flatten the columns
pivot_df = pivot_df.reset_index()
pivot_df = pivot_df.set_index(('filenames',"",""))
pivot_df = pivot_df.rename_axis(index = {('filenames',"",""):'filenames'})

In [ ]:
pivot_df


In [ ]:
# Function to calculate the differences and add columns
def add_difference_columns(df):
    areas = df.columns.get_level_values('area').unique()
    freqs = df.columns.get_level_values('freq').unique()

    for area in areas:
        for freq in freqs:
            
            before_col = (area, freq, '1_Before')
            during_col = (area, freq, '2_During')
            after_col = (area, freq, '3_After')
            
            diff_during_before = df[during_col] - df[before_col]
            diff_after_before = df[after_col] - df[before_col]

            diff_col_name_during_before = (area,freq,'4_DiffDuringBefore')
            diff_col_name_after_before =  (area,freq,'5_DiffAfterBefore')

            df[diff_col_name_during_before] = diff_during_before
            df[diff_col_name_after_before] = diff_after_before

    return df

# Call the function to add difference columns
df_with_diff = add_difference_columns(pivot_df)


In [ ]:
def sort_columns(df):
    # Sort the columns based on the specified rule
    sorted_columns = sorted(df.columns, key=lambda col: (col[0], col[1], col[2]))

    # Reindex the DataFrame with the sorted columns
    sorted_df = df.reindex(columns=sorted_columns)

    return sorted_df


df_with_diff_sorted = sort_columns(df_with_diff)
df_with_diff_sorted.columns

In [ ]:
areas = df_with_diff_sorted.columns.get_level_values('area').unique()
areas

In [25]:
# save files
df_with_diff_sorted.to_excel(osp.join(folder_name,f'lfp_rawdata_{sociability}.xlsx'))